# Jour 1 - Exercice 4 : Manipulation Avancée et Visualisation de Données

## Objectifs
- Maîtriser les techniques avancées de manipulation de données avec Pandas
- Créer des visualisations interactives et informatives avec Plotly
- Réaliser une analyse exploratoire approfondie
- Découvrir des insights pertinents dans les données de satisfaction des passagers
- Apprendre à combiner plusieurs visualisations dans un tableau de bord

## Introduction

Dans ce notebook, nous allons approfondir les techniques de manipulation de données et de visualisation. Nous utiliserons le jeu de données de satisfaction des passagers d'une compagnie aérienne pour mettre en pratique ces concepts avancés. Ce notebook est conçu pour consolider et étendre les connaissances acquises dans les exercices précédents.

## 1. Préparation de l'environnement et chargement des données

Commençons par importer les bibliothèques nécessaires et charger notre jeu de données.

In [ ]:
# Importation des bibliothèques standard
import pandas as pd
import numpy as np

# Bibliothèques de visualisation
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuration pour un meilleur affichage
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 100)

# Pour les calculs statistiques
from scipy import stats

# Pour les palettes de couleurs personnalisées
import plotly.colors as pc

print(f"Pandas version: {pd.__version__}")
print(f"Plotly version: {plotly.__version__}")

In [ ]:
# Chargement du jeu de données
df = pd.read_csv('../../data/passenger_satisfaction/train.csv')

# Affichage des premières lignes
df.head()

## 2. Exploration et nettoyage avancés des données

Avant de procéder à l'analyse, examinons la structure de nos données et effectuons un nettoyage approfondi.

In [ ]:
# Informations sur le DataFrame
df.info()

In [ ]:
# Vérification des valeurs manquantes
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

# Créer un DataFrame pour mieux visualiser les valeurs manquantes
missing_df = pd.DataFrame({
    'Nombre de valeurs manquantes': missing_values,
    'Pourcentage (%)': missing_percentage
})

# Afficher uniquement les colonnes avec des valeurs manquantes
missing_df[missing_df['Nombre de valeurs manquantes'] > 0].sort_values('Pourcentage (%)', ascending=False)

In [ ]:
# Vérification des valeurs manquantes
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

# Créer un DataFrame pour mieux visualiser les valeurs manquantes
missing_df = pd.DataFrame({
    'Nombre de valeurs manquantes': missing_values,
    'Pourcentage (%)': missing_percentage
})

# Afficher uniquement les colonnes avec des valeurs manquantes
missing_df[missing_df['Nombre de valeurs manquantes'] > 0].sort_values('Pourcentage (%)', ascending=False)

### 2.2 Traitement avancé des valeurs manquantes

Nous allons traiter les valeurs manquantes de manière plus sophistiquée en utilisant différentes stratégies selon la nature des colonnes.

In [ ]:
# Créer une copie du DataFrame pour ne pas modifier l'original
df_clean = df.copy()

# 1. Pour 'Arrival Delay in Minutes', nous utiliserons une imputation conditionnelle
# Calculons d'abord la corrélation entre 'Departure Delay in Minutes' et 'Arrival Delay in Minutes'
correlation = df['Departure Delay in Minutes'].corr(df['Arrival Delay in Minutes'].dropna())
print(f"Corrélation entre retard de départ et d'arrivée: {correlation:.4f}")

# Si la corrélation est forte, nous pouvons utiliser une régression linéaire simple pour prédire les valeurs manquantes
if abs(correlation) > 0.7:  # Seuil arbitraire pour une forte corrélation
    # Données pour la régression
    X = df.dropna(subset=['Arrival Delay in Minutes'])[['Departure Delay in Minutes']]
    y = df.dropna(subset=['Arrival Delay in Minutes'])['Arrival Delay in Minutes']
    
    # Régression linéaire simple
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    model.fit(X, y)
    
    # Prédire les valeurs manquantes
    missing_indices = df['Arrival Delay in Minutes'].isna()
    if missing_indices.sum() > 0:
        predictions = model.predict(df[missing_indices][['Departure Delay in Minutes']])
        df_clean.loc[missing_indices, 'Arrival Delay in Minutes'] = predictions
        print(f"Valeurs imputées pour 'Arrival Delay in Minutes': {len(predictions)}")
else:
    # Si la corrélation n'est pas forte, utiliser la médiane
    df_clean['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].median(), inplace=True)
    print(f"Valeurs manquantes dans 'Arrival Delay in Minutes' remplacées par la médiane")

# 2. Pour les autres colonnes avec des valeurs manquantes (si présentes)
# Identifier les colonnes numériques et catégorielles
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Imputer les valeurs manquantes dans les colonnes numériques avec la médiane
for col in numeric_cols:
    if df[col].isna().sum() > 0 and col != 'Arrival Delay in Minutes':  # Déjà traité
        df_clean[col].fillna(df[col].median(), inplace=True)
        print(f"Valeurs manquantes dans '{col}' remplacées par la médiane")

# Imputer les valeurs manquantes dans les colonnes catégorielles avec le mode
for col in categorical_cols:
    if df[col].isna().sum() > 0:
        df_clean[col].fillna(df[col].mode()[0], inplace=True)
        print(f"Valeurs manquantes dans '{col}' remplacées par le mode")

# Vérifier qu'il n'y a plus de valeurs manquantes
print(f"\nNombre total de valeurs manquantes restantes: {df_clean.isna().sum().sum()}")

### 2.3 Transformation et création de nouvelles variables

Créons de nouvelles variables qui pourraient révéler des insights intéressants.

In [ ]:
df.head()

In [ ]:
# 1. Catégorisation de l'âge
df_clean['Age_Group'] = pd.cut(
    df_clean['Age'],
    bins=[0, 18, 25, 35, 50, 65, 100],
    labels=['<18', '18-25', '26-35', '36-50', '51-65', '65+'],
    right=False
)

# 2. Calcul du retard total (départ + arrivée)
df_clean['Total_Delay'] = df_clean['Departure Delay in Minutes'] + df_clean['Arrival Delay in Minutes']

# 3. Catégorisation du retard total
df_clean['Delay_Category'] = pd.cut(
    df_clean['Total_Delay'],
    bins=[-float('inf'), 0, 15, 60, float('inf')],
    labels=['No Delay', 'Short Delay', 'Medium Delay', 'Long Delay']
)

# 4. Score moyen de satisfaction (pour toutes les colonnes de rating)
# Identifier les colonnes de rating (celles qui contiennent 'rating' dans leur nom)
rating_cols = ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']
df_clean['Average_Rating'] = df_clean[rating_cols].mean(axis=1)

# 5. Écart-type des ratings (pour mesurer la cohérence des évaluations)
df_clean['Rating_StdDev'] = df_clean[rating_cols].std(axis=1)

# 6. Créer une variable binaire pour la satisfaction
df_clean['Is_Satisfied'] = df_clean['Satisfaction'].map({'satisfied': 1, 'neutral or dissatisfied': 0})

# 7. Créer une variable pour la durée du vol par tranche de 100 km
df_clean['Flight_Duration_per_100km'] = df_clean['Flight Distance'] / 100

# Afficher les nouvelles colonnes
df_clean[['Age', 'Age_Group', 'Total_Delay', 'Delay_Category', 'Average_Rating', 'Rating_StdDev', 'Is_Satisfied', 'Flight_Duration_per_100km']].head()

### 2.4 Analyse des corrélations avancée

Examinons les corrélations entre les variables numériques et leur relation avec la satisfaction.

In [ ]:
# Sélectionner les colonnes numériques pertinentes pour l'analyse de corrélation
numeric_cols_for_corr = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 
                         'Arrival Delay in Minutes', 'Total_Delay', 'Average_Rating', 
                         'Rating_StdDev', 'Is_Satisfied']
numeric_cols_for_corr.extend(rating_cols)

# Calculer la matrice de corrélation
corr_matrix = df_clean[numeric_cols_for_corr].corr()

# Créer une heatmap interactive avec Plotly
fig = px.imshow(
    corr_matrix,
    text_auto='.2f',
    aspect="auto",
    color_continuous_scale='RdBu_r',  # Échelle de couleur rouge-bleu inversée
    title="Matrice de corrélation des variables numériques",
    zmin=-1, zmax=1  # Limites pour la corrélation
)

fig.update_layout(height=800, width=900)
fig.show()

### 2.5 Analyse des variables les plus corrélées avec la satisfaction

In [ ]:
# Extraire les corrélations avec la satisfaction et les trier
satisfaction_corr = corr_matrix['Is_Satisfied'].drop('Is_Satisfied').sort_values(ascending=False)

# Créer un graphique à barres horizontal pour visualiser les corrélations
fig = px.bar(
    x=satisfaction_corr.values,
    y=satisfaction_corr.index,
    orientation='h',
    title="Corrélation des variables avec la satisfaction",
    labels={'x': 'Coefficient de corrélation', 'y': 'Variables'},
    color=satisfaction_corr.values,
    color_continuous_scale='RdBu_r',
    range_color=[-1, 1]
)

fig.update_traces(texttemplate='%{x:.3f}', textposition='outside')
fig.update_layout(height=600, width=800)
fig.show()

In [ ]:
# Extraire les corrélations avec la satisfaction et les trier
satisfaction_corr = corr_matrix['Is_Satisfied'].drop('Is_Satisfied').sort_values(ascending=False)

# Créer un graphique à barres horizontal pour visualiser les corrélations
fig = px.bar(
    x=satisfaction_corr.values,
    y=satisfaction_corr.index,
    orientation='h',
    title="Corrélation des variables avec la satisfaction",
    labels={'x': 'Coefficient de corrélation', 'y': 'Variables'},
    color=satisfaction_corr.values,
    color_continuous_scale='RdBu_r',
    range_color=[-1, 1]
)

fig.update_traces(texttemplate='%{x:.3f}', textposition='outside')
fig.update_layout(height=600, width=1000)
fig.show()

## 3. Visualisations avancées

Maintenant que nous avons nettoyé et transformé nos données, créons des visualisations plus avancées pour explorer les facteurs qui influencent la satisfaction des passagers.

### 3.1 Distribution des évaluations par niveau de satisfaction

Comparons la distribution des évaluations entre les passagers satisfaits et insatisfaits.

In [ ]:
# Créer un dataframe avec les colonnes de rating et la satisfaction
rating_df = df_clean[rating_cols + ['Satisfaction']]

# Convertir de format wide à format long pour faciliter la visualisation
rating_long = pd.melt(rating_df, 
                      id_vars=['Satisfaction'], 
                      value_vars=rating_cols,
                      var_name='Service', 
                      value_name='Rating')

# Créer un boxplot pour comparer les distributions
fig = px.box(rating_long, 
             x='Service', 
             y='Rating', 
             color='Satisfaction',
             title='Distribution des évaluations par service et niveau de satisfaction',
             color_discrete_map={'satisfied': '#2ca02c', 'neutral or dissatisfied': '#d62728'},
             category_orders={'Service': sorted(rating_long['Service'].unique())},
             labels={'Rating': 'Évaluation (0-5)', 'Service': 'Service', 'Satisfaction': 'Satisfaction'}
            )

fig.update_layout(height=600, width=1000, boxmode='group')
fig.update_xaxes(tickangle=45)
fig.show()

### 3.2 Analyse multidimensionnelle avec des graphiques à bulles

Créons un graphique à bulles pour visualiser plusieurs dimensions simultanément.

In [ ]:
# Calculer les moyennes par groupe d'âge et classe
bubble_data = df_clean.groupby(['Age_Group', 'Class'])['Is_Satisfied'].mean().reset_index()
bubble_data.rename(columns={'Is_Satisfied': 'Satisfaction_Rate'}, inplace=True)

# Calculer le nombre de passagers dans chaque groupe
counts = df_clean.groupby(['Age_Group', 'Class']).size().reset_index(name='Count')
bubble_data = bubble_data.merge(counts, on=['Age_Group', 'Class'])

# Calculer la note moyenne pour chaque groupe
avg_ratings = df_clean.groupby(['Age_Group', 'Class'])['Average_Rating'].mean().reset_index()
bubble_data = bubble_data.merge(avg_ratings, on=['Age_Group', 'Class'])

# Convertir le taux de satisfaction en pourcentage
bubble_data['Satisfaction_Rate'] = bubble_data['Satisfaction_Rate'] * 100

# Créer le graphique à bulles
fig = px.scatter(bubble_data, 
                x='Age_Group', 
                y='Satisfaction_Rate', 
                size='Count', 
                color='Class',
                hover_name='Class',
                hover_data=['Average_Rating', 'Count'],
                text='Class',
                size_max=60,
                title='Taux de satisfaction par groupe d\'âge et classe',
                labels={'Age_Group': 'Groupe d\'âge', 
                        'Satisfaction_Rate': 'Taux de satisfaction (%)', 
                        'Count': 'Nombre de passagers',
                        'Class': 'Classe',
                        'Average_Rating': 'Évaluation moyenne'}
               )

fig.update_traces(textposition='top center')
fig.update_layout(height=600, width=900)
fig.update_yaxes(range=[0, 100])
fig.show()

### 3.3 Analyse des retards et leur impact sur la satisfaction

Explorons comment les retards affectent la satisfaction des passagers.

In [ ]:
# Créer des bins pour les retards de départ
df_clean['Departure_Delay_Bins'] = pd.cut(
    df_clean['Departure Delay in Minutes'],
    bins=[-1, 0, 15, 30, 60, 120, float('inf')],
    labels=['Pas de retard', '1-15 min', '16-30 min', '31-60 min', '1-2 heures', '> 2 heures']
)

# Calculer le taux de satisfaction pour chaque bin de retard et type de voyage
delay_impact = df_clean.groupby(['Departure_Delay_Bins', 'Type of Travel'])['Is_Satisfied'].mean().reset_index()
delay_impact.rename(columns={'Is_Satisfied': 'Satisfaction_Rate'}, inplace=True)
delay_impact['Satisfaction_Rate'] = delay_impact['Satisfaction_Rate'] * 100

# Créer un graphique linéaire
fig = px.line(delay_impact, 
              x='Departure_Delay_Bins', 
              y='Satisfaction_Rate', 
              color='Type of Travel',
              markers=True,
              title='Impact du retard au départ sur la satisfaction par type de voyage',
              labels={'Departure_Delay_Bins': 'Retard au départ', 
                      'Satisfaction_Rate': 'Taux de satisfaction (%)', 
                      'Type of Travel': 'Type de voyage'}
             )

fig.update_layout(height=500, width=900)
fig.update_yaxes(range=[0, 100])
fig.show()

### 3.4 Analyse des facteurs de satisfaction par classe

Créons un radar chart pour comparer les évaluations moyennes par classe.

In [ ]:
# Calculer les moyennes des évaluations par classe
class_ratings = df_clean.groupby('Class')[rating_cols].mean().reset_index()

# Préparer les données pour le radar chart
categories = [col.replace('_rating', '').replace('_', ' ').title() for col in rating_cols]

# Créer le radar chart
fig = go.Figure()

for i, class_type in enumerate(class_ratings['Class'].unique()):
    values = class_ratings[class_ratings['Class'] == class_type][rating_cols].values.flatten().tolist()
    # Ajouter la première valeur à la fin pour fermer le polygone
    values.append(values[0])
    categories_closed = categories + [categories[0]]
    
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=categories_closed,
        fill='toself',
        name=class_type
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 5]
        )),
    showlegend=True,
    title='Évaluations moyennes par classe et service',
    height=600,
    width=800
)

fig.show()

### 3.5 Analyse des segments de clientèle

Utilisons une visualisation en treemap pour explorer les segments de clientèle et leur satisfaction.

In [ ]:
# Créer un dataframe agrégé par type de client, type de voyage et classe
segment_data = df_clean.groupby(['Customer Type', 'Type of Travel', 'Class']).agg(
    Count=('ID', 'count'),
    Satisfaction_Rate=('Is_Satisfied', 'mean'),
    Avg_Rating=('Average_Rating', 'mean')
).reset_index()

# Convertir le taux de satisfaction en pourcentage
segment_data['Satisfaction_Rate'] = segment_data['Satisfaction_Rate'] * 100

# Créer une colonne pour l'affichage dans le treemap
segment_data['Label'] = segment_data['Customer Type'] + '<br>' + \
                        segment_data['Type of Travel'] + '<br>' + \
                        segment_data['Class'] + '<br>' + \
                        'Satisfaction: ' + segment_data['Satisfaction_Rate'].round(1).astype(str) + '%'

# Créer le treemap
fig = px.treemap(
    segment_data,
    path=['Customer Type', 'Type of Travel', 'Class'],
    values='Count',
    color='Satisfaction_Rate',
    hover_data=['Avg_Rating'],
    color_continuous_scale='RdYlGn',
    range_color=[0, 100],
    title='Segments de clientèle et taux de satisfaction',
    labels={'Count': 'Nombre de passagers', 'Satisfaction_Rate': 'Taux de satisfaction (%)'}
)

fig.update_traces(textinfo='label+value+percent parent')
fig.update_layout(height=700, width=900)
fig.show()

## 4. Tableau de bord interactif

Créons un tableau de bord interactif qui combine plusieurs visualisations pour une analyse plus complète.

In [ ]:
# Créer un tableau de bord avec plusieurs sous-graphiques
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Satisfaction par groupe d\'âge',
        'Satisfaction par classe et type de voyage',
        'Impact du retard sur la satisfaction',
        'Top 5 facteurs influençant la satisfaction'
    ),
    specs=[
        [{'type': 'bar'}, {'type': 'pie'}],
        [{'type': 'scatter'}, {'type': 'bar'}]
    ],
    vertical_spacing=0.1,
    horizontal_spacing=0.1
)

# 1. Graphique à barres: Satisfaction par groupe d'âge
age_satisfaction = df_clean.groupby('Age_Group')['Is_Satisfied'].mean().reset_index()
age_satisfaction['Satisfaction_Rate'] = age_satisfaction['Is_Satisfied'] * 100

fig.add_trace(
    go.Bar(
        x=age_satisfaction['Age_Group'],
        y=age_satisfaction['Satisfaction_Rate'],
        text=age_satisfaction['Satisfaction_Rate'].round(1),
        textposition='auto',
        marker_color='#1f77b4'
    ),
    row=1, col=1
)

# 2. Graphique circulaire: Satisfaction par classe et type de voyage
class_travel_satisfaction = df_clean.groupby(['Class', 'Type of Travel']).agg(
    Count=('ID', 'count'),
    Satisfaction_Rate=('Is_Satisfied', 'mean')
).reset_index()
class_travel_satisfaction['Satisfaction_Rate'] = class_travel_satisfaction['Satisfaction_Rate'] * 100
class_travel_satisfaction['Label'] = class_travel_satisfaction['Class'] + ' - ' + class_travel_satisfaction['Type of Travel']

fig.add_trace(
    go.Pie(
        labels=class_travel_satisfaction['Label'],
        values=class_travel_satisfaction['Count'],
        text=class_travel_satisfaction['Satisfaction_Rate'].round(1).astype(str) + '%',
        hoverinfo='label+text+percent',
        textinfo='percent',
        marker=dict(colors=px.colors.qualitative.Pastel)
    ),
    row=1, col=2
)

# 3. Graphique linéaire: Impact du retard sur la satisfaction
delay_bins = pd.cut(
    df_clean['Total_Delay'],
    bins=[-float('inf'), 0, 15, 30, 60, 120, float('inf')],
    labels=['0', '1-15', '16-30', '31-60', '61-120', '>120']
)
df_clean['Delay_Bins'] = delay_bins
delay_impact = df_clean.groupby('Delay_Bins')['Is_Satisfied'].mean().reset_index()
delay_impact['Satisfaction_Rate'] = delay_impact['Is_Satisfied'] * 100

fig.add_trace(
    go.Scatter(
        x=delay_impact['Delay_Bins'],
        y=delay_impact['Satisfaction_Rate'],
        mode='lines+markers+text',
        text=delay_impact['Satisfaction_Rate'].round(1),
        textposition='top center',
        line=dict(color='#2ca02c', width=3),
        marker=dict(size=10)
    ),
    row=2, col=1
)

# 4. Graphique à barres: Top 5 facteurs influençant la satisfaction
top_factors = satisfaction_corr.head(5)

fig.add_trace(
    go.Bar(
        x=top_factors.values,
        y=top_factors.index,
        orientation='h',
        text=top_factors.values.round(3),
        textposition='auto',
        marker_color='#d62728'
    ),
    row=2, col=2
)

# Mise à jour de la mise en page
fig.update_layout(
    title_text='Tableau de bord de la satisfaction des passagers',
    height=900,
    width=1200,
    showlegend=False
)

# Mise à jour des axes
fig.update_yaxes(title_text='Taux de satisfaction (%)', range=[0, 100], row=1, col=1)
fig.update_xaxes(title_text='Groupe d\'âge', row=1, col=1)

fig.update_yaxes(title_text='Taux de satisfaction (%)', range=[0, 100], row=2, col=1)
fig.update_xaxes(title_text='Retard total (minutes)', row=2, col=1)

fig.update_yaxes(title_text='Facteurs', row=2, col=2)
fig.update_xaxes(title_text='Coefficient de corrélation', row=2, col=2)

fig.show()

## 5. Exercices pratiques

### Exercice 1: Analyse des passagers fréquents vs occasionnels

Objectif: Comparer la satisfaction et les évaluations des passagers fréquents (Loyal Customer) 
et occasionnels (disloyal Customer)
1. Créez un graphique comparant les taux de satisfaction entre ces deux groupes
2. Analysez si les facteurs de satisfaction diffèrent entre ces deux groupes
3. Visualisez les différences d'évaluation pour chaque service

In [39]:
# Votre code ici

### Exercice 2: Impact de la distance de vol sur la satisfaction

Objectif: Analyser comment la distance de vol influence la satisfaction des passagers
1. Créez des catégories de distance (court, moyen et long courrier)
2. Visualisez le taux de satisfaction par catégorie de distance
3. Analysez si certains services sont plus importants selon la distance

In [40]:
# Votre code ici

### Exercice 3: Création d'un score de prédiction de satisfaction

Objectif: Créer un score simple pour prédire la satisfaction des passagers
1. Sélectionnez les 3-5 variables les plus corrélées à la satisfaction
2. Créez une formule pondérée basée sur ces variables
3. Évaluez la performance de votre score en calculant son taux de précision

In [ ]:
# Votre code ici

### Exercice 4: Analyse personnalisée

Objectif: Réalisez une analyse de votre choix qui n'a pas encore été couverte

Quelques idées:
- Analyse par genre et âge
- Impact combiné du type de voyage et de la classe
- Analyse temporelle (si des données de date sont disponibles)
- Segmentation avancée des clients

In [ ]:
# Votre code ici

## 6. Conclusion et recommandations

Dans ce notebook, nous avons réalisé une analyse approfondie des données de satisfaction des passagers aériens. Voici les principales conclusions et recommandations :

### Principaux facteurs influençant la satisfaction :
1. **Confort et services à bord** : Les évaluations des services en vol sont fortement corrélées à la satisfaction globale.
2. **Retards** : Les retards ont un impact négatif significatif sur la satisfaction, particulièrement pour les voyageurs d'affaires.
3. **Classe de voyage** : Les passagers de la classe affaires sont généralement plus satisfaits que ceux de la classe économique.
4. **Segments de clientèle** : Nous avons identifié plusieurs segments distincts avec des besoins et des attentes différents.

### Recommandations pour améliorer la satisfaction :
1. **Améliorer les services en vol** : Se concentrer sur les facteurs ayant la plus forte corrélation avec la satisfaction.
2. **Réduire les retards** : Mettre en place des mesures pour minimiser les retards, particulièrement pour les vols d'affaires.
3. **Personnaliser l'expérience** : Adapter les services en fonction des différents segments de clientèle identifiés.
4. **Améliorer la communication** : Informer proactivement les passagers en cas de retard pour atténuer l'impact sur la satisfaction.